# PCR-GLOBWB 2

In [1]:
from os import environ, remove
from configparser import ConfigParser
from glob import glob
from pathlib import Path

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from cartopy import crs
from cftime import num2date

import hydrostats.metrics as hm

import pickle

from grpc4bmi.bmi_client_docker import BmiClientDocker
from ewatercycle.observation.grdc import get_grdc_data

In [2]:
def var_to_xarray(model, variable):
    # Get grid properties from model (x = latitude !!)
    # could be speedup, lots of bmi calls are done here that dont change between updates
    shape = model.get_grid_shape(model.get_var_grid(variable))
    lat = model.get_grid_x(model.get_var_grid(variable))
    lon = model.get_grid_y(model.get_var_grid(variable))
    time = num2date(model.get_current_time(), model.get_time_units())

    # Get model data for variable at current timestep
    data = model.get_value(variable)
    data = np.reshape(data, shape)

    # Create xarray object
    da = xr.DataArray(data, 
                      coords = {'longitude': lon, 'latitude': lat, 'time': time}, 
                      dims = ['latitude', 'longitude'],
                      name = variable,
                      attrs = {'units': model.get_var_units(variable)}
                     )

    # Masked invalid values on return array:
    return da.where(da != -999)

In [3]:
# currently global data are available on cartesius
input_dir='/mnt/data/examples/technical_paper/pcr-globwb/input/'
output_dir='/mnt/home/user43/Sophie_Bep/output2'
forcing_dir = '/mnt/data/examples/technical_paper/pcr-globwb/input/'

setting_file = '/mnt/home/user43/Sophie_Bep/Sophie_rhine_05min_era5.ini'

In [4]:
model = BmiClientDocker(image='ewatercycle/pcrg-grpc4bmi:latest', image_port=55555,

                   input_dir=input_dir,

                   output_dir=output_dir)

 

model.initialize(setting_file)

In [5]:
output = []
time_units = model.get_time_units()
print(f'Running pcrglobwb', flush=True)
while model.get_current_time() < model.get_end_time():
    time = num2date(model.get_current_time(), time_units)
    print(f'Current time: {time}', end="\r")
    model.update()
    output.append(var_to_xarray(model, variable="total_evaporation"))
print('')

Running pcrglobwb
Current time: 2014-12-30 00:00:00


In [6]:
data  = xr.concat(output, dim='time')
data

<xarray.DataArray 'total_evaporation' (time: 5844, latitude: 108, longitude: 126)>
array([[[2.51154270e-04, 2.97485531e-04, 2.76457082e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [2.75817176e-04, 2.56088853e-04, 1.50178996e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [2.76811246e-04, 2.40933688e-04, 1.87580605e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         9.62761260e-05, 9.96188392e-05, 1.00093675e-04],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.21493860e-04, 1.03859187e-04, 9.43305713e-05],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         9.68468739e-05, 9.93087087e-05, 8.66143673e-05]],

       [[2.51556281e-04, 2.97730003e-04, 2.76790815e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [2.75756291e-04, 2.56041909e-04, 1.50093998e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [2.76881154e-04, 2.41292670e-04, 1.87857207e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         9.65589279e-05, 9.99141776e-05, 1.00383040e-04],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.21868674e-04, 1.04174382e-04, 9.46068612e-05],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         9.71339177e-05, 9.96149320e-05, 8.68682619e-05]],

       [[2.52028171e-04, 2.98150349e-04, 2.77029554e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [2.75400846e-04, 2.55724735e-04, 1.49904648e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [2.76749372e-04, 2.41663889e-04, 1.88134552e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         9.68674285e-05, 1.00236015e-04, 1.00699144e-04],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.22275043e-04, 1.04517501e-04, 9.49085297e-05],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         9.74472205e-05, 9.99479889e-05, 8.71456141e-05]],

       ...,

       [[1.44748890e-04, 1.72215339e-04, 1.58313691e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.56545953e-04, 1.45861370e-04, 8.46562616e-05, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.56842449e-04, 1.37619252e-04, 1.06969012e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         8.10000711e-05, 8.39863787e-05, 8.46843905e-05],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.03610029e-04, 8.90135270e-05, 8.07899269e-05],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         8.25869502e-05, 8.38696142e-05, 7.28691593e-05]],

       [[1.80788789e-04, 2.13575986e-04, 1.98153255e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.94403401e-04, 1.81385331e-04, 1.06500367e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.95720539e-04, 1.72569213e-04, 1.34568822e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.06876316e-04, 1.10655092e-04, 1.11773094e-04],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.35117589e-04, 1.17301402e-04, 1.06624735e-04],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.08981389e-04, 1.10434521e-04, 9.61994010e-05]],

       [[1.65548845e-04, 1.91399900e-04, 1.83022246e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.75120091e-04, 1.64389785e-04, 1.00257304e-04, ...,
         0.00000000e+00, 0.00000000e+00, 0.0000

In [7]:
BE_BRA = data.sel(latitude=51.30761, longitude=4.51984, method='nearest')*1000
ts_BRA = BE_BRA.to_dataframe()

BE_VIE = data.sel(latitude=50.30493, longitude=5.99812, method='nearest')*1000
ts_VIE = BE_VIE.to_dataframe()

NL_LOO = data.sel(latitude=52.16658, longitude=5.74356, method='nearest')*1000
ts_LOO = NL_LOO.to_dataframe()

CH_DAV = data.sel(latitude=46.81533, longitude=9.85591, method='nearest')*1000
ts_DAV = CH_DAV.to_dataframe()

IT_REN = data.sel(latitude=46.58686, longitude=11.43369, method='nearest')*1000
ts_REN = IT_REN.to_dataframe()

DE_GEB = data.sel(latitude=51.09973, longitude=10.91463, method='nearest')*1000
ts_GEB = DE_GEB.to_dataframe()

In [8]:
ts_BRA.to_csv(r'/mnt/home/user43/Sophie_Bep/Rijn/PCRGLOB_evap_BE_BRA.csv')
ts_VIE.to_csv(r'/mnt/home/user43/Sophie_Bep/Rijn/PCRGLOB_evap_BE_VIE.csv')
ts_LOO.to_csv(r'/mnt/home/user43/Sophie_Bep/Rijn/PCRGLOB_evap_NL_LOO.csv')
ts_DAV.to_csv(r'/mnt/home/user43/Sophie_Bep/Rijn/PCRGLOB_evap_CH_DAV.csv')
ts_REN.to_csv(r'/mnt/home/user43/Sophie_Bep/Rijn/PCRGLOB_evap_IT_REN.csv')
ts_GEB.to_csv(r'/mnt/home/user43/Sophie_Bep/Rijn/PCRGLOB_evap_DE_GEB.csv')

In [9]:
tsdag_BRA = ts_BRA["2004-01-01": '2014-12-31']
tsdag_VIE = ts_VIE["2002-01-01": '2014-12-31']
tsdag_LOO = ts_LOO["2002-01-01": '2014-12-31']
tsdag_DAV = ts_DAV["2006-01-01": '2014-12-31']
tsdag_REN = ts_REN["2006-01-01": '2014-12-31']
tsdag_GEB = ts_GEB["2002-01-01": '2014-12-31']

In [10]:
Flux_file1 = pd.read_csv("/mnt/home/user43/Sophie_Bep/Rijn/FLX_BE-Bra_FLUXNET2015_SUBSET_DD_1996-2014_2-4.csv", skipinitialspace=True, parse_dates=[0],index_col=0)
Flux_file2 = pd.read_csv("/mnt/home/user43/Sophie_Bep/Rijn/FLX_BE-Vie_FLUXNET2015_SUBSET_DD_1996-2014_1-4.csv", skipinitialspace=True, parse_dates=[0],index_col=0)
Flux_file3 = pd.read_csv("/mnt/home/user43/Sophie_Bep/Rijn/FLX_NL-Loo_FLUXNET2015_SUBSET_DD_1996-2014_1-4.csv", skipinitialspace=True, parse_dates=[0],index_col=0)
Flux_file4 = pd.read_csv("/mnt/home/user43/Sophie_Bep/Rijn/FLX_CH-Dav_FLUXNET2015_SUBSET_DD_1997-2014_1-4.csv", skipinitialspace=True, parse_dates=[0],index_col=0)
Flux_file5 = pd.read_csv("/mnt/home/user43/Sophie_Bep/Rijn/FLX_IT-Ren_FLUXNET2015_SUBSET_DD_1998-2013_1-4.csv", skipinitialspace=True, parse_dates=[0],index_col=0)
Flux_file6 = pd.read_csv("/mnt/home/user43/Sophie_Bep/Rijn/FLX_DE-Geb_FLUXNET2015_SUBSET_DD_2001-2014_1-4.csv", skipinitialspace=True, parse_dates=[0],index_col=0)


In [11]:
lamda = 2.43 * (10 **6)
rho = 1000

In [12]:
Flux_file1 = Flux_file1[["LE_F_MDS"]] / (lamda * rho) * 86400000
Flux_file2 = Flux_file2[["LE_F_MDS"]] / (lamda * rho) * 86400000
Flux_file3 = Flux_file3[["LE_F_MDS"]] / (lamda * rho) * 86400000
Flux_file4 = Flux_file4[["LE_F_MDS"]] / (lamda * rho) * 86400000
Flux_file5 = Flux_file5[["LE_F_MDS"]] / (lamda * rho) * 86400000
Flux_file6 = Flux_file6[["LE_F_MDS"]] / (lamda * rho) * 86400000

In [13]:
Fluxnet_BRA = Flux_file1["2004-01-01": '2014-12-31']["LE_F_MDS"]
Fluxnet_VIE = Flux_file2["2002-01-01": '2014-12-31']["LE_F_MDS"]
Fluxnet_LOO = Flux_file3["2002-01-01": '2014-12-31']["LE_F_MDS"]
Fluxnet_DAV = Flux_file4["2006-01-01": '2014-12-31']["LE_F_MDS"]
Fluxnet_REN = Flux_file5["2006-01-01": '2014-12-31']["LE_F_MDS"]
Fluxnet_GEB = Flux_file6["2002-01-01": '2014-12-31']["LE_F_MDS"]

In [14]:
tsdag_BRA = tsdag_BRA.join(Fluxnet_BRA)
tsdag_VIE = tsdag_VIE.join(Fluxnet_VIE)
tsdag_LOO = tsdag_LOO.join(Fluxnet_LOO)
tsdag_DAV = tsdag_DAV.join(Fluxnet_DAV)
tsdag_REN = tsdag_REN.join(Fluxnet_REN)
tsdag_GEB = tsdag_GEB.join(Fluxnet_GEB)

In [15]:
tsdag_BRA.to_csv(r'/mnt/home/user43/Sophie_Bep/Rijn/evap_BE_BRA.csv')
tsdag_VIE.to_csv(r'/mnt/home/user43/Sophie_Bep/Rijn/evap_BE_VIE.csv')
tsdag_LOO.to_csv(r'/mnt/home/user43/Sophie_Bep/Rijn/evap_NL_LOO.csv')
tsdag_DAV.to_csv(r'/mnt/home/user43/Sophie_Bep/Rijn/evap_CH_DAV.csv')
tsdag_REN.to_csv(r'/mnt/home/user43/Sophie_Bep/Rijn/evap_IT_REN.csv')
tsdag_GEB.to_csv(r'/mnt/home/user43/Sophie_Bep/Rijn/evap_DE_GEB.csv')

In [16]:
model.finalize()